# Limpeza das planilhas da BanVic

Para trabalhar com as planilhas no software de visualização, é preciso verificar os dados se estão 100% consistentes sem valores nulos e fazer as devidas limpezas. No caso, foi feita uma varredura preliminar no excel para averiguar se não havia duplicatas lançadas.

A qualidade das planilhas está bem sólida no geral.

# 1. Pacotes e Funções

In [159]:
import pandas as pd
from datetime import datetime, timezone, timedelta

### 1.2 Funções

Função para converter a data em uma string na qual será lida facilmente pelo PowerBI

In [178]:
def converter_string_em_data(data):
    if len(data) <= 10:
        data = datetime.strptime(data, "%Y-%m-%d")
        data_convertida = data.strftime("%d/%m/%Y")
        return data_convertida
    elif len(data) <= 23:
        data = datetime.strptime(data, "%Y-%m-%d %H:%M:%S %Z")
        data_utc = data.replace(tzinfo=timezone.utc)
        data_brt = data_utc.astimezone(timezone(timedelta(hours=-3)))   
        data_convertida = data_brt.strftime("%d/%m/%Y %H:%M:%S")
        return data_convertida
    else:
        data = datetime.strptime(data, "%Y-%m-%d %H:%M:%S.%f %Z")
        data_utc = data.replace(tzinfo=timezone.utc)
        data_brt = data_utc.astimezone(timezone(timedelta(hours=-3)))   
        data_convertida = data_brt.strftime("%d/%m/%Y %H:%M:%S")
        return data_convertida


# print(converter_string_em_data("2022-05-22"))
# print(converter_string_em_data("2020-05-22 03:15:21 UTC"))
# print(converter_string_em_data('2022-12-30 00:00:00.041757 UTC'))

Funções para recuperar o UF do endereço e excluir o UF deste mesmo endereço

In [179]:
def recuperar_uf(endereco):
    return endereco[-2:]
def deletar_uf(endereco):
    return endereco[:-5]

# endereco = "Estação da Cunha, 64 Vila Santo Antônio Barroquinha 42157-327 Teixeira de Farias / RS"
# print(recuperar_uf(endereco))
# print(deletar_uf(endereco))

Remoção do Hifen caso precise no campo CEP

In [177]:
def remover_hifen(cep):
    if len(cep) <= 8:
        return cep
    else:
        cep_novo = cep.split("-")
        cep = f"{cep_novo[0]}{cep_novo[1]}"
        return cep

# print(remover_hifen("05326-333"))
# print(remover_hifen("53133999"))

05326333
53133999


# 2. Dataframes

### 2.1. Agências

In [119]:
# Leitura do arquivo agencias em CSV com o Pandas
agencias_df = pd.read_csv("./csv/agencias.csv",sep=",")
agencias_df.head()


,cod_agencia,nome,endereco,cidade,uf,data_abertura,tipo_agencia
0,7,Agência Digital,"Av. Paulista, 1436 - Cerqueira César, São Paul...",São Paulo,SP,2015-08-01,Digital
1,1,Agência Matriz,"Av. Paulista, 1436 - Cerqueira César, São Paul...",São Paulo,SP,2010-01-01,Física
2,2,Agência Tatuapé,"Praça Sílvio Romero, 158 - Tatuapé, São Paulo ...",São Paulo,SP,2010-06-14,Física
3,3,Agência Campinas,"Av. Francisco Glicério, 895 - Vila Lidia, Camp...",Campinas,SP,2012-03-04,Física
4,4,Agência Osasco,"Av. Antônio Carlos Costa, 1000 - Bela Vista, O...",Osasco,SP,2013-11-06,Física


In [120]:
# Conversão da coluna data_abertura em um formato válido para o PowerBI
agencias_df["data_abertura"] = agencias_df["data_abertura"].apply(converter_string_em_data)
agencias_df.head(12)

,cod_agencia,nome,endereco,cidade,uf,data_abertura,tipo_agencia
0,7,Agência Digital,"Av. Paulista, 1436 - Cerqueira César, São Paul...",São Paulo,SP,01/08/2015,Digital
1,1,Agência Matriz,"Av. Paulista, 1436 - Cerqueira César, São Paul...",São Paulo,SP,01/01/2010,Física
2,2,Agência Tatuapé,"Praça Sílvio Romero, 158 - Tatuapé, São Paulo ...",São Paulo,SP,14/06/2010,Física
3,3,Agência Campinas,"Av. Francisco Glicério, 895 - Vila Lidia, Camp...",Campinas,SP,04/03/2012,Física
4,4,Agência Osasco,"Av. Antônio Carlos Costa, 1000 - Bela Vista, O...",Osasco,SP,06/11/2013,Física
5,5,Agência Porto Alegre,"Av. Bento Gonçalves, 1924 - Partenon, Porto Al...",Porto Alegre,RS,01/12/2013,Física
6,6,Agência Rio de Janeiro,"R. Sen. Dantas, 15 - Centro, Rio de Janeiro - ...",Rio de Janeiro,RJ,01/04/2015,Física
7,8,Agência Jardins,"Av. Brg. Faria Lima, 2491 - Jardim Paulistano,...",São Paulo,SP,09/01/2018,Física
8,9,Agência Florianópolis,"Av. Jorn. Rubéns de Arruda Ramos, 1280 - Centr...",Florianópolis,SC,09/10/2019,Física
9,10,Agência Recife,"Av. Conselheiro Aguiar, 4432 - Boa Viagem, Rec...",Recife,PE,09/10/2021,Física


In [121]:
# Exportação do arquivo em formato Excel
agencias_df.to_excel("./excel/agencias.xlsx",index=False)

### 2.2. Clientes

In [196]:
# Leitura do arquivo agencias em CSV com o Pandas
clientes_df = pd.read_csv("./csv/clientes.csv",sep=",")
clientes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 998 entries, 0 to 997
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   cod_cliente      998 non-null    int64 
 1   primeiro_nome    998 non-null    object
 2   ultimo_nome      998 non-null    object
 3   email            998 non-null    object
 4   tipo_cliente     998 non-null    object
 5   data_inclusao    998 non-null    object
 6   cpfcnpj          998 non-null    object
 7   data_nascimento  998 non-null    object
 8   endereco         998 non-null    object
 9   cep              998 non-null    object
dtypes: int64(1), object(9)
memory usage: 78.1+ KB


In [197]:
clientes_df.head(5)

,cod_cliente,primeiro_nome,ultimo_nome,email,tipo_cliente,data_inclusao,cpfcnpj,data_nascimento,endereco,cep
0,28,Sabrina,Dias,moreiraemanuelly@example.org,PF,2017-04-03 16:11:00 UTC,357.081.496-39,2006-08-11,Praia de Duarte Vila Piratininga 81327-166 Fer...,95140-704
1,674,Luiz Felipe,Dias,pedroferreira@example.org,PF,2021-02-10 13:27:00 UTC,085.196.374-93,1995-10-11,"Avenida da Rosa, 654 João Paulo Ii 20295449 Nu...",76516-765
2,693,Renan,Dias,ogomes@example.net,PF,2020-01-21 13:12:00 UTC,783.416.059-10,1948-11-19,Jardim de Rodrigues Ipiranga 14161-477 Duarte ...,51779625
3,743,Clara,Dias,rafaelcorreia@example.org,PF,2019-05-06 11:39:00 UTC,589.237.610-95,1978-01-27,"Colônia Thomas Silva, 9 Tupi B 15771-946 Ferna...",19615792
4,769,André,Dias,danilo33@example.org,PF,2017-01-07 14:53:00 UTC,459.608.721-02,1990-08-25,"Rua Correia, 889 Diamante 59123250 Aragão / RS",01672838


In [198]:
# Conversão da coluna data_inclusao e data_nascimento em um formato válido para o PowerBI
clientes_df["data_nascimento"] = clientes_df["data_nascimento"].apply(converter_string_em_data)
clientes_df["data_inclusao"] = clientes_df["data_inclusao"].apply(converter_string_em_data)
clientes_df.head(5)


,cod_cliente,primeiro_nome,ultimo_nome,email,tipo_cliente,data_inclusao,cpfcnpj,data_nascimento,endereco,cep
0,28,Sabrina,Dias,moreiraemanuelly@example.org,PF,03/04/2017 13:11:00,357.081.496-39,11/08/2006,Praia de Duarte Vila Piratininga 81327-166 Fer...,95140-704
1,674,Luiz Felipe,Dias,pedroferreira@example.org,PF,10/02/2021 10:27:00,085.196.374-93,11/10/1995,"Avenida da Rosa, 654 João Paulo Ii 20295449 Nu...",76516-765
2,693,Renan,Dias,ogomes@example.net,PF,21/01/2020 10:12:00,783.416.059-10,19/11/1948,Jardim de Rodrigues Ipiranga 14161-477 Duarte ...,51779625
3,743,Clara,Dias,rafaelcorreia@example.org,PF,06/05/2019 08:39:00,589.237.610-95,27/01/1978,"Colônia Thomas Silva, 9 Tupi B 15771-946 Ferna...",19615792
4,769,André,Dias,danilo33@example.org,PF,07/01/2017 11:53:00,459.608.721-02,25/08/1990,"Rua Correia, 889 Diamante 59123250 Aragão / RS",01672838


In [199]:
# Criação de colunas para estado separada do endereço
clientes_df["uf"] = clientes_df["endereco"].apply(recuperar_uf)
clientes_df["endereco"] = clientes_df["endereco"].apply(deletar_uf)
clientes_df.head(5)

,cod_cliente,primeiro_nome,ultimo_nome,email,tipo_cliente,data_inclusao,cpfcnpj,data_nascimento,endereco,cep,uf
0,28,Sabrina,Dias,moreiraemanuelly@example.org,PF,03/04/2017 13:11:00,357.081.496-39,11/08/2006,Praia de Duarte Vila Piratininga 81327-166 Fer...,95140-704,SP
1,674,Luiz Felipe,Dias,pedroferreira@example.org,PF,10/02/2021 10:27:00,085.196.374-93,11/10/1995,"Avenida da Rosa, 654 João Paulo Ii 20295449 Nunes",76516-765,AP
2,693,Renan,Dias,ogomes@example.net,PF,21/01/2020 10:12:00,783.416.059-10,19/11/1948,Jardim de Rodrigues Ipiranga 14161-477 Duarte ...,51779625,RN
3,743,Clara,Dias,rafaelcorreia@example.org,PF,06/05/2019 08:39:00,589.237.610-95,27/01/1978,"Colônia Thomas Silva, 9 Tupi B 15771-946 Ferna...",19615792,SP
4,769,André,Dias,danilo33@example.org,PF,07/01/2017 11:53:00,459.608.721-02,25/08/1990,"Rua Correia, 889 Diamante 59123250 Aragão",01672838,RS


In [200]:
# Remoção do hífen da coluna cep
clientes_df["cep"] = clientes_df["cep"].apply(remover_hifen)
clientes_df.head(5)

,cod_cliente,primeiro_nome,ultimo_nome,email,tipo_cliente,data_inclusao,cpfcnpj,data_nascimento,endereco,cep,uf
0,28,Sabrina,Dias,moreiraemanuelly@example.org,PF,03/04/2017 13:11:00,357.081.496-39,11/08/2006,Praia de Duarte Vila Piratininga 81327-166 Fer...,95140704,SP
1,674,Luiz Felipe,Dias,pedroferreira@example.org,PF,10/02/2021 10:27:00,085.196.374-93,11/10/1995,"Avenida da Rosa, 654 João Paulo Ii 20295449 Nunes",76516765,AP
2,693,Renan,Dias,ogomes@example.net,PF,21/01/2020 10:12:00,783.416.059-10,19/11/1948,Jardim de Rodrigues Ipiranga 14161-477 Duarte ...,51779625,RN
3,743,Clara,Dias,rafaelcorreia@example.org,PF,06/05/2019 08:39:00,589.237.610-95,27/01/1978,"Colônia Thomas Silva, 9 Tupi B 15771-946 Ferna...",19615792,SP
4,769,André,Dias,danilo33@example.org,PF,07/01/2017 11:53:00,459.608.721-02,25/08/1990,"Rua Correia, 889 Diamante 59123250 Aragão",01672838,RS


In [201]:
# Exportação do arquivo em formato Excel
clientes_df.to_excel("./excel/clientes.xlsx",index=False)

### 2.3. tabela associativa - Colaboradores e Agência

In [109]:
colaborador_agencia = pd.read_csv("./csv/colaborador_agencia.csv",sep=",")
colaborador_agencia.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   cod_colaborador  100 non-null    int64
 1   cod_agencia      100 non-null    int64
dtypes: int64(2)
memory usage: 1.7 KB


In [ ]:
# Exportação do arquivo em formato Excel
colaborador_agencia.to_excel("./excel/colaborador_agencia.xlsx",index=False)

### 2.4. Colaboradores

In [202]:
# Conversão da coluna data_inclusao e data_nascimento em um formato válido para o PowerBI
colaboradores_df = pd.read_csv("./csv/colaboradores.csv",sep=",")
colaboradores_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   cod_colaborador  100 non-null    int64 
 1   primeiro_nome    100 non-null    object
 2   ultimo_nome      100 non-null    object
 3   email            100 non-null    object
 4   cpf              100 non-null    object
 5   data_nascimento  100 non-null    object
 6   endereco         100 non-null    object
 7   cep              100 non-null    object
dtypes: int64(1), object(7)
memory usage: 6.4+ KB


In [203]:
# Conversão da coluna data_nascimento em um formato válido para o PowerBI
colaboradores_df["data_nascimento"] = colaboradores_df["data_nascimento"].apply(converter_string_em_data)
colaboradores_df.head(12)

,cod_colaborador,primeiro_nome,ultimo_nome,email,cpf,data_nascimento,endereco,cep
0,14,Paulo,Dias,melissalopes@example.net,847.210.695-02,24/04/1974,"Lagoa de Rodrigues, 62 Zilah Sposito 37328-273...",27275674
1,19,Luiz Fernando,Dias,pcunha@example.net,820.415.963-33,10/07/1994,"Ladeira de Moreira, 82 Jardim Atlântico 27617-...",50013962
2,32,Vitor Hugo,Dias,aragaonicolas@example.net,936.172.548-37,18/12/1986,Área Sophie Moreira Maria Virgínia 71941-111 S...,08955-215
3,88,João Gabriel,Lima,luiz-otaviorezende@example.org,967.082.451-67,19/10/1994,"Trevo Murilo Ramos, 92 São Benedito 73615-597 ...",24224-305
4,39,Maria Eduarda,Melo,joao-pedro72@example.com,857.496.210-49,01/03/1996,"Parque de Moura, 65 Granja Werneck 67769059 da...",96896390
5,41,Olivia,Costa,sabrina59@example.com,190.768.435-20,30/01/1966,"Favela Rocha, 94 Laranjeiras 92500120 Viana / GO",90019993
6,45,Laura,Cunha,lviana@example.org,217.583.096-95,07/02/1977,"Lagoa Benício Moura, 5 Vila Paraíso 50235854 d...",19415-341
7,100,Lavínia,Cunha,noahmoraes@example.com,062.937.158-02,24/03/1975,"Colônia Lima, 83 Flamengo 36172-665 da Mota / SE",74890752
8,16,Benjamin,Gomes,vjesus@example.org,718.950.436-20,21/12/1963,"Conjunto Gomes, 888 Céu Azul 76308-427 Lopes d...",29604844
9,46,Alana,Gomes,samuel76@example.com,768.134.950-20,16/07/1977,Fazenda Emilly Costa Campo Alegre 55916-800 da...,88161-168


In [204]:
# Criação de colunas para estado separada do endereço
colaboradores_df["uf"] = colaboradores_df["endereco"].apply(recuperar_uf)
colaboradores_df["endereco"] = colaboradores_df["endereco"].apply(deletar_uf)
colaboradores_df.head(5)

,cod_colaborador,primeiro_nome,ultimo_nome,email,cpf,data_nascimento,endereco,cep,uf
0,14,Paulo,Dias,melissalopes@example.net,847.210.695-02,24/04/1974,"Lagoa de Rodrigues, 62 Zilah Sposito 37328-273...",27275674,PE
1,19,Luiz Fernando,Dias,pcunha@example.net,820.415.963-33,10/07/1994,"Ladeira de Moreira, 82 Jardim Atlântico 27617-...",50013962,PB
2,32,Vitor Hugo,Dias,aragaonicolas@example.net,936.172.548-37,18/12/1986,Área Sophie Moreira Maria Virgínia 71941-111 S...,08955-215,RJ
3,88,João Gabriel,Lima,luiz-otaviorezende@example.org,967.082.451-67,19/10/1994,"Trevo Murilo Ramos, 92 São Benedito 73615-597 ...",24224-305,RR
4,39,Maria Eduarda,Melo,joao-pedro72@example.com,857.496.210-49,01/03/1996,"Parque de Moura, 65 Granja Werneck 67769059 da...",96896390,RR


In [205]:
# Remoção do hífen da coluna cep
colaboradores_df["cep"] = colaboradores_df["cep"].apply(remover_hifen)
colaboradores_df.head(5)

,cod_colaborador,primeiro_nome,ultimo_nome,email,cpf,data_nascimento,endereco,cep,uf
0,14,Paulo,Dias,melissalopes@example.net,847.210.695-02,24/04/1974,"Lagoa de Rodrigues, 62 Zilah Sposito 37328-273...",27275674,PE
1,19,Luiz Fernando,Dias,pcunha@example.net,820.415.963-33,10/07/1994,"Ladeira de Moreira, 82 Jardim Atlântico 27617-...",50013962,PB
2,32,Vitor Hugo,Dias,aragaonicolas@example.net,936.172.548-37,18/12/1986,Área Sophie Moreira Maria Virgínia 71941-111 S...,08955215,RJ
3,88,João Gabriel,Lima,luiz-otaviorezende@example.org,967.082.451-67,19/10/1994,"Trevo Murilo Ramos, 92 São Benedito 73615-597 ...",24224305,RR
4,39,Maria Eduarda,Melo,joao-pedro72@example.com,857.496.210-49,01/03/1996,"Parque de Moura, 65 Granja Werneck 67769059 da...",96896390,RR


In [206]:
# Exportação do arquivo em formato Excel
colaboradores_df.to_excel("./excel/colaboradores.xlsx",index=False)

### 2.5. Contas

In [137]:
# Leitura do arquivo agencias em CSV com o Pandas
contas_df = pd.read_csv("./csv/contas.csv",sep=",")
contas_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999 entries, 0 to 998
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   num_conta               999 non-null    int64  
 1   cod_cliente             999 non-null    int64  
 2   cod_agencia             999 non-null    int64  
 3   cod_colaborador         999 non-null    int64  
 4   tipo_conta              999 non-null    object 
 5   data_abertura           999 non-null    object 
 6   saldo_total             999 non-null    float64
 7   saldo_disponivel        999 non-null    float64
 8   data_ultimo_lancamento  999 non-null    object 
dtypes: float64(2), int64(4), object(3)
memory usage: 70.4+ KB


In [138]:
# Conversão da coluna data_abertura e data_ultimo_lancamento em um formato válido para o PowerBI
contas_df["data_abertura"] = contas_df["data_abertura"].apply(converter_string_em_data)
contas_df["data_ultimo_lancamento"] = contas_df["data_ultimo_lancamento"].apply(converter_string_em_data)
contas_df.head(12)

,num_conta,cod_cliente,cod_agencia,cod_colaborador,tipo_conta,data_abertura,saldo_total,saldo_disponivel,data_ultimo_lancamento
0,53,53,1,7,PF,24/05/2011 12:30:00,2984.7615,2814.67,02/07/2019 08:03:46
1,190,190,1,7,PF,17/10/2014 11:48:00,4262.4322,4191.86,25/03/2021 09:00:15
2,191,191,1,7,PF,23/03/2013 11:14:00,3266.5680,3080.71,09/09/2016 20:34:15
3,203,203,1,7,PF,03/08/2016 13:13:00,21241.5424,19461.56,28/12/2022 04:58:24
4,254,254,1,7,PF,24/10/2012 10:48:00,13335.6966,13197.54,20/12/2013 19:20:00
5,306,306,1,7,PF,23/05/2012 09:11:00,22997.4738,21677.88,05/10/2016 00:14:36
6,472,472,1,7,PF,07/08/2022 09:08:00,4178.4237,4183.68,29/12/2022 21:00:00
7,520,520,1,7,PF,03/11/2019 13:26:00,1809.8087,1288.51,29/12/2022 20:59:59
8,739,739,1,7,PF,26/02/2014 09:20:00,45.7691,616.79,29/12/2022 20:59:59
9,509,509,1,13,PF,20/01/2012 08:56:00,19159.7952,19863.76,09/06/2016 19:25:04


In [139]:
# Exportação do arquivo em formato Excel
contas.to_excel("./excel/contas.xlsx",index=False)

### 2.6. Propostas de Crédito

In [145]:
propostas_credito_df = pd.read_csv("./csv/propostas_credito.csv",sep=",")
propostas_credito_df.head()



,cod_proposta,cod_cliente,cod_colaborador,data_entrada_proposta,taxa_juros_mensal,valor_proposta,valor_financiamento,valor_entrada,valor_prestacao,quantidade_parcelas,carencia,status_proposta
0,116,338,1,2014-05-30 18:43:12 UTC,0.0194,36199.950355,50032.03,13832.079645,1045.219825,58,0,Enviada
1,715,45,1,2021-06-17 13:09:32 UTC,0.0131,12897.538285,19848.55,6951.011715,232.124685,100,1,Enviada
2,755,494,1,2021-08-10 04:24:02 UTC,0.0203,122236.312551,187154.36,64918.047449,3673.518951,56,0,Enviada
3,953,381,1,2011-02-23 07:57:58 UTC,0.0167,47784.649757,59530.61,11745.960243,955.041560,109,3,Enviada
4,1046,176,1,2010-11-15 16:03:28 UTC,0.0211,178393.038243,214294.19,35901.151757,10219.659415,22,4,Enviada


In [146]:
# Conversão da coluna data_entrada_proposta em um formato válido para o PowerBI
propostas_credito_df["data_entrada_proposta"] = propostas_credito_df["data_entrada_proposta"].apply(converter_string_em_data)
propostas_credito_df.head(12)

,cod_proposta,cod_cliente,cod_colaborador,data_entrada_proposta,taxa_juros_mensal,valor_proposta,valor_financiamento,valor_entrada,valor_prestacao,quantidade_parcelas,carencia,status_proposta
0,116,338,1,30/05/2014 15:43:12,0.0194,36199.950355,50032.03,13832.079645,1045.219825,58,0,Enviada
1,715,45,1,17/06/2021 10:09:32,0.0131,12897.538285,19848.55,6951.011715,232.124685,100,1,Enviada
2,755,494,1,10/08/2021 01:24:02,0.0203,122236.312551,187154.36,64918.047449,3673.518951,56,0,Enviada
3,953,381,1,23/02/2011 04:57:58,0.0167,47784.649757,59530.61,11745.960243,955.041560,109,3,Enviada
4,1046,176,1,15/11/2010 13:03:28,0.0211,178393.038243,214294.19,35901.151757,10219.659415,22,4,Enviada
5,1171,732,1,04/05/2020 18:51:58,0.0242,40476.959008,56464.88,15987.920992,20976.064001,2,0,Enviada
6,1434,560,1,30/01/2020 21:02:49,0.0169,137637.615490,192947.94,55310.324510,2861.607605,100,0,Enviada
7,1744,159,1,04/03/2016 17:31:58,0.0180,1317.125762,1605.63,288.504238,36.078665,60,2,Enviada
8,1864,776,1,13/09/2019 07:21:31,0.0110,106127.470610,128472.96,22345.489390,1774.933849,98,4,Enviada
9,1928,421,1,08/05/2014 12:43:27,0.0200,3455.606489,6668.98,3213.373511,211.333552,20,2,Enviada


In [147]:
# Exportação do arquivo em formato Excel
propostas_credito_df.to_excel("./excel/propostas_credito.xlsx",index=False)

### 2.7. Transações

In [113]:
# Leitura do arquivo transacoes em CSV com o Pandas
transacoes_df = pd.read_csv("./csv/transacoes.csv",sep=",")
transacoes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71999 entries, 0 to 71998
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   cod_transacao    71999 non-null  int64  
 1   num_conta        71999 non-null  int64  
 2   data_transacao   71999 non-null  object 
 3   nome_transacao   71999 non-null  object 
 4   valor_transacao  71999 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 2.7+ MB


In [114]:
# Conversão da coluna data_transação em um formato válido para o PowerBI
transacoes_df["data_transacao"] = transacoes_df["data_transacao"].apply(converter_string_em_data)
transacoes_df.head(12)

,cod_transacao,num_conta,data_transacao,nome_transacao,valor_transacao
0,3100,256,22/05/2020 00:15:21,Saque,-130.0
1,28405,256,29/06/2021 08:52:38,Saque,-255.0
2,1776,512,17/05/2020 10:04:50,Saque,-250.0
3,14311,512,05/09/2020 23:55:34,Saque,-80.0
4,25012,512,16/10/2020 06:26:52,Saque,-475.0
5,26883,512,08/11/2020 21:47:27,Saque,-210.0
6,35017,512,13/04/2021 05:17:24,Saque,-50.0
7,46264,512,01/05/2021 03:25:50,Saque,-1160.0
8,21340,768,14/02/2020 18:42:15,Saque,-170.0
9,64259,1,11/08/2019 12:13:07,Saque,-245.0


In [115]:
# Exportação do arquivo em formato Excel
transacoes_df.to_excel("./excel/transacoes.xlsx",index=False)